# Proyecto Bimestral: Sistema de Recuperación de Información basado en Imágenes
Nombres:
Cesar Duque
Geovanny Luna

## 1. Introducción
El objetivo de este trabajo es diseñar y desarrollar una máquina de búsqueda que permita a los usuarios realizar consultas utilizando imágenes en lugar de texto. Este sistema debe ser capaz de encontrar imágenes similares dentro de una base de datos dada. El proyecto se dividirá en varias fases, que se describen a continuación.

## 2. Fases del Proyecto

### 2.1. Adquisición de Datos
**Objetivo:** Obtener y preparar el dataset Caltech101.  
**Tareas:**
- Descargar el dataset.
- Descomprimir y organizar los archivos.



Agregamos las librerias

In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Descargamos el dataset para guardar el train en train_dataset y la data de test en test_dataset y tambien informacion de la data en dataset_info

In [2]:
(train_dataset, test_dataset), dataset_info = tfds.load(
    name='caltech101',
    split=['train', 'test'],
    with_info=True,
    as_supervised=True,
    data_dir='C:/Users/cesar/Downloads/tensorflow_datasets',
    download=False
)

In [7]:
name_classes = dataset_info.features['label'].names
pd.DataFrame(name_classes)

,0
0,accordion
1,airplanes
2,anchor
3,ant
4,background_google
...,...
97,wheelchair
98,wild_cat
99,windsor_chair
100,wrench


### 2.2. Preprocesamiento
**Objetivo:** Preparar los datos para su análisis.  
**Tareas:**
- Usar técnicas de preprocesamiento de imágenes, como normalización, reducción de tamaño, o eliminación de ruido.
- Documentar cada paso del preprocesamiento.

Se crea la funcion para preprocesar las imagenes a un tamaño de 224x224 y se normaliza sus valores

In [8]:
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

Se ejecuta la funcion para procesar las imagenes de los datos del train y del test

In [9]:
train_dataset = train_dataset.map(preprocess_image).shuffle(1000).batch(32)
test_dataset = test_dataset.map(preprocess_image).batch(32)

In [10]:
print(train_dataset)
print(test_dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


### 2.3. Extracción de Características
**Objetivo:** Extraer las características de las imágenes en una forma que los algoritmos puedan procesar.  
**Tareas:**
- Utilizar un modelo de red neuronal convolucional (CNN) para extraer características de las imágenes.
- Entrenar un modelo desde cero o utilizar un modelo preentrenado y aplicar transfer learning.
- Documentar los métodos y resultados obtenidos.


In [11]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model   

In [12]:
# Load the VGG16 model with pretrained weights from ImageNet, without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a new model that outputs the feature maps
model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)

In [13]:
# Function to extract features
def extract_features(dataset):
    features = []
    labels = []
    imgs = []
    for images, lbls in dataset:
        imgs.append(images)
        feature_maps = model.predict(images)
        features.append(feature_maps)
        labels.append(lbls.numpy())
    return features, labels,imgs

In [14]:
# Extract features for train and test datasets
train_features, train_labels,train_img = extract_features(train_dataset)
test_features, test_labels,test_img = extract_features(test_dataset)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 



### 2.4. Indexación
**Objetivo:** Crear un índice que permita búsquedas eficientes.  
**Tareas:**
- Desarrollar un sistema para indexar las características extraídas de las imágenes y permitir la búsqueda eficiente de imágenes similares.
- Considerar técnicas como k-NN o índices especializados como KD-Trees o LSH (Locality-Sensitive Hashing).
- Documentar el proceso de construcción del índice.

### 2.5. Diseño del Motor de Búsqueda
**Objetivo:** Implementar la funcionalidad de búsqueda.  
**Tareas:**
- Desarrollar la lógica para procesar consultas de usuarios a partir de imágenes.
- Desarrollar un algoritmo de ranking para ordenar los resultados.
- Documentar la arquitectura y los algoritmos utilizados.

### 2.6. Evaluación del Sistema
**Objetivo:** Medir la efectividad del sistema.  
**Tareas:**
- Definir un conjunto de métricas de evaluación (precisión, recall, F1-score).
- Definir un benchmark para evaluación del sistema.
- Comparar el rendimiento de diferentes configuraciones del sistema.
- Documentar los resultados y análisis.

### 2.7. Interfaz Web de Usuario
**Objetivo:** Crear una interfaz para interactuar con el sistema.  
**Tareas:**
- Diseñar una interfaz web donde los usuarios puedan subir una imagen y recibir resultados con imágenes similares.
- Mostrar los resultados de búsqueda de manera clara y ordenada.
- La interfaz debe ser intuitiva y facilitar la interacción con el sistema.
- Documentar el diseño y funcionalidades de la interfaz.

## 3. Entrega Final
- **Documentación Completa:** Incluyendo los procesos, decisiones tomadas, y resultados de cada fase.
- **Código Fuente:** Organizado y bien comentado.
- **Informe de Evaluación:** Análisis detallado de la evaluación del sistema.
- **Demostración del Sistema:** Presentación funcional del sistema a través de la interfaz web
